I modify and exclude various code chunks from the original code here (e.g removing extra semantic filtering of candidates, graph features).
* Modify  to get multiple entities per text, not just first : `get_multiple_nel` instead of `test_nel`

In [1]:
import os
# Set the environment variable
# os.environ['NX_CUGRAPH_AUTOCONFIG'] = 'True'
# %env NX_CUGRAPH_AUTOCONFIG=True
## make networkx use gpu
## https://developer.nvidia.com/blog/networkx-introduces-zero-code-change-acceleration-using-nvidia-cugraph/

In [2]:
#!/usr/bin/env python
# coding: utf-8
import pandas as pd
import numpy as np
from IPython import get_ipython
from IPython.core.display_functions import display
from tqdm import tqdm  # Import tqdm for the progress bar

## scispacy, medspacy, medcat, quickumls, semrep...
import networkx as nx
from itertools import combinations

import spacy
import scispacy
from itertools import compress
from scispacy.linking import EntityLinker
from scispacy.abbreviation import AbbreviationDetector ## https://github.com/allenai/scispacy?tab=readme-ov-file#example-usage

import sys, os ## append parent path to dir to allow import
parent_directory = os.path.abspath('..')
sys.path.append(parent_directory)
from util import get_sentence_pairs_similarity, wrangle_df_icu, anti_join_df,test_nel,get_multiple_nel,link_kg_concepts_slim
from util import *
import string
rem = string.punctuation
punct_pattern = r"[{}]".format(rem)
# get_ipython().run_line_magic('load_ext', 'autoreload')
# get_ipython().run_line_magic('autoreload', '2')
from configs import * #config_gall
import logging
import warnings
# warnings.filterwarnings('ignore', message=".*UserWarning*.")
# warnings.filterwarnings('ignore', message='.*Duplicate section*.')

# Suppress specific warnings
warnings.simplefilter("ignore", UserWarning)  # Catch UserWarnings more broadly
warnings.filterwarnings("ignore", message=".*The component 'matcher' does not have any patterns defined.*")

# Set up logger to suppress spaCy's warnings
logger = logging.getLogger("spacy")
logger.setLevel(logging.ERROR)

In [3]:
cutoffYear = 2012
FastRun = False#True
SAVE = False # True

In [4]:
# def test_nel(text,nlp,do_print=False):
#     doc = nlp(text)
#     # Let's look at a random entity!
#     if do_print: print("All ents", doc.ents)
#     ## broken code snippet : https://github.com/allenai/scispacy/issues/355
#     for e in doc.ents:
#         if e._.kb_ents:
#             cui = e._.kb_ents[0][0]
#             print(e, cui)

#     if do_print:print("\n--------------------------\n")
#     entity = doc.ents[0]
#     if do_print:print("Name: ", entity)

#     # Each entity is linked to UMLS with a score
#     # (currently just char-3gram matching).
#     if do_print:
#         linker = nlp.get_pipe("scispacy_linker")
#         for umls_ent in entity._.kb_ents:
#             print(linker.kb.cui_to_entity[umls_ent[0]])
#     return entity

# def get_multiple_nel(text, nlp,do_print=False):
#     "modified to get multiple entities , not just first"
#     doc = nlp(text)
#     if do_print:print("All entities:", doc.ents)
    
#     entities_with_cuis = []
    
#     for e in doc.ents:
#         if e._.kb_ents:
#             cuis = [cui[0] for cui in e._.kb_ents]
#             if do_print:print(f"Entity: {e}, CUIs: {cuis}")
#             entities_with_cuis.append({
#                 'entity': e,
#                 'cuis': cuis
#             })
    
#     if do_print:
#         print("\n--------------------------\n")
#         linker = nlp.get_pipe("scispacy_linker")
#         for ent in entities_with_cuis:
#             print(f"Details for Entity: {ent['entity']}")
#             for umls_ent in ent['cuis']:
#                 print(linker.kb.cui_to_entity[umls_ent])
    
#     return entities_with_cuis


In [5]:
# ## slim in that some parts disabled/commented out (semantic similarity filter etc'): renamed from link_kg_concepts       
# def link_kg_concepts_slim(FEATURES_REPORT_PATH, CANDIDATE_NOVEL_CUIS_FILEPATH, TARGET_NAME, additional_target_cui_terms_list=[], SAVE_OUTPUTS = False, MIN_EVIDENCE_FILTER = 1,
#                      REMOVE_CUI_TERMS_LIST=['Prieto syndrome', "Polarized Reflectance Spectroscopy",
#                                             # mistaken extraction from PRS - drop it for now for cleanliness
#                                             'Standard (qualifier)', 'Standard base excess calculation technique',
#                                             'Standard of Care', 'Spatial Frequency', 'Disease',
#                                             'Statistical Frequency', 'Kind of quantity - Frequency', 'Concentration measurement',
#                                             'Concentration measurement', 'Illness (finding)', 'Concentration Ratio',
#                                             "Special", "ActInformationPrivacyReason <operations>", "Left sided", "Left", "Right",
#                                             "Table Cell Horizontal Align - left", "Query Quantity Unit - Records", "Up",
#                                             "Qualification",
#                                             "Visit", "Total", "Participant", "Overall", "Right sided", "Left sided",
#                                             "Take", "Percent (qualifier value)","Population Group",
#                                             "Diagnosis","Coding","Code",
#                                             "Average" , "Comparison" , "Lost" , "Yes - Presence findings", 
#                                             "Pharmaceutical Preparations","Physicians","Mother (person)","Father (person)","Severe (severity modifier)",
#                                             ]
#                      , input_kg_path="../../SemMed/predications.parquet", EXCLUDE_TUIS_LIST = ["T079", "T093", "T094", "T095", "T170", "T204", "T201", "T065",
#                          "T078", ], sem_similarity_threshhold_score=0.12, # 0.15
#                      # top_cutoff_simMin = 0.39,top_cutoff_kgHit = 2,
#                      FAST=False,
#                     return_df=True):
#     # global nlp, df_kg_sep, df_hits, G
#     global df_kg_sep, df_hits ## maybe disable this..

#     # ### Load processed semmed db
#     # * ths version has 1 row per triple.
#     # * `Count` is the number of unique PMIDs the triple has appeared in
#     df_kg = pd.read_parquet(input_kg_path)

#     # ### Filtered version

#     df_kg = df_kg.loc[df_kg["pair_counts"] >= MIN_EVIDENCE_FILTER].reset_index(drop=True).copy()
#     df_kg.drop(columns=["pair_counts","counts"], inplace=True, errors="ignore")
#     print("After filtering KG min count")
#     for c in df_kg.select_dtypes("category").columns:
#         # remove unobserved categories, in new filtered data
#         df_kg[c] = df_kg[c].cat.remove_unused_categories()
#     print(df_kg.shape[0])  # 4M

#     # # !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_scibert-0.5.4.tar.gz
#     # # !pip install 'spacy[transformers]'
#     # ```
#     # there's also lg, and transformer based
#     if FAST:
#         nlp = spacy.load("en_core_sci_sm")
#     else:
#         nlp = spacy.load("en_core_sci_lg")
#         # nlp = spacy.load("en_core_sci_scibert") # error in later spacy version?
    
#     # This line takes a while, because we have to download ~1GB of data
#     # and load a large JSON file (the knowledge base). 
    
#     # Add the abbreviation pipe to the spacy pipeline. (if using resolve_abbreviations )
#     nlp.add_pipe("abbreviation_detector")
#     nlp.add_pipe("scispacy_linker",
#                  config={"resolve_abbreviations": True,
#                          "linker_name": "umls",
#                          "max_entities_per_mention": 2  # 3, #6, #4, #5
#                      , "threshold": 0.82  ## default is 0.8, paper mentions 0.99 as thresh
#                          })

#     print("TARGET_NAME (For Entity-KG linking", TARGET_NAME)
#     # new func
#     # entity = get_multiple_nel(TARGET_NAME,nlp=nlp)
#     entity = test_nel(TARGET_NAME,nlp=nlp)
#     # #### Target terms - may manually change
#     ## list of extracted CUIs meaning sepsis. Note that not all correct even here and with threshhold
#     list_target_cuis = [i[0] for i in entity._.kb_ents]
#     if len(additional_target_cui_terms_list) > 0:
#         list_target_cuis = list(set(list_target_cuis + additional_target_cui_terms_list))
#     print("list_target_cuis",list_target_cuis)
#     # #### get subset of KG with target in pairs
#     # (not 100% sure if ideal, but will save time when comparing features )
#     df_kg_sep = df_kg.loc[(df_kg["SUBJECT_CUI"].isin(list_target_cuis)) | (df_kg["OBJECT_CUI"].isin(list_target_cuis))].copy()
#     df_kg_sep.drop_duplicates(['SUBJECT_CUI', 'SUBJECT_NAME', 'OBJECT_CUI', 'OBJECT_NAME'],
#                               inplace=True)  # ignore predicate type for this filter table
#     # In[28]:
#     ## sample set of terms from gallstone prediction + filtered
#     icu_feature_terms = pd.read_csv(FEATURES_REPORT_PATH)

#     if FAST:
#         icu_feature_terms = icu_feature_terms.loc[icu_feature_terms["feature_importance"] > 0].head(25)
#     icu_feature_terms = wrangle_df_icu(icu_feature_terms)
#     ##############################################################
#     ##ORIG code deeted,  , single etity per text - deleted
#     ##...
#     ##############################################################
#     ## slt/new - for multiple entities per text
    
#     # Initialize lists to collect data
#     novel_cols_candidates_names = []
#     no_entities_list = []
#     novel_candidate_cuis = []
#     novel_candidate_cuis_nomenclatures = []
#     TUIs_list = []
#     list_cui_kg_hits = []
#     list_cui_definitions = []
    
#     # Iterate over each feature name
#     for f in icu_feature_terms["name"]:
#         doc = nlp(f)
#         linker = nlp.get_pipe("scispacy_linker")
        
#         # Check if any entities are found in the feature name
#         if doc.ents:
#             # Flag to check if at least one CUI passes the TUI filter
#             feature_has_valid_cui = False
            
#             # Iterate over all entities in the document
#             for entity in doc.ents:
#                 # Extract all CUIs for the current entity
#                 list_feature_cuis = [cui[0] for cui in entity._.kb_ents] if entity._.kb_ents else []
                
#                 if not list_feature_cuis:
#                     continue  # Skip entities without CUIs
                
#                 # Apply TUI filter to exclude certain CUIs
#                 tui_filter_mask = [
#                     linker.kb.cui_to_entity[c][3][0] not in EXCLUDE_TUIS_LIST 
#                     for c in list_feature_cuis
#                 ]
#                 filtered_cuis = list(compress(list_feature_cuis, tui_filter_mask))
                
#                 # If no CUIs pass the TUI filter, skip to the next entity
#                 if not filtered_cuis:
#                     continue
                
#                 # Mark that this feature has at least one valid CUI
#                 feature_has_valid_cui = True
                
#                 # Extract nomenclatures for the filtered CUIs
#                 list_cuis_nomenclatures = [
#                     linker.kb.cui_to_entity[c][1] for c in filtered_cuis
#                 ]
                
#                 # Append TUIs for the filtered CUIs
#                 TUIs_list.extend([linker.kb.cui_to_entity[c][3][0] for c in filtered_cuis])
                
#                 # Filter the knowledge graph for related CUIs
#                 df_related = df_kg_sep.loc[
#                     df_kg_sep["SUBJECT_CUI"].isin(filtered_cuis) | 
#                     df_kg_sep["OBJECT_CUI"].isin(filtered_cuis)
#                 ]
                
#                 # Iterate over each filtered CUI
#                 for cui in filtered_cuis:
#                     # Count KG hits for the current CUI
#                     ## add .loc
#                     num_kg_hits = df_related.loc[
#                         (df_kg_sep["SUBJECT_CUI"] == cui) | 
#                         (df_related["OBJECT_CUI"] == cui)
#                     ].shape[0]
#                     list_cui_kg_hits.append(num_kg_hits)
                    
#                     # Collect definitions and synonyms for the current CUI
#                     cui_definition = linker.kb.cui_to_entity[cui][4]
#                     cui_def_text = f"{linker.kb.cui_to_entity[cui][1]}. {cui_definition}" if cui_definition else linker.kb.cui_to_entity[cui][1]
#                     list_cui_definitions.append(cui_def_text)
                
#                 # Extend lists with the current feature name and associated CUIs
#                 novel_cols_candidates_names.extend([f] * len(filtered_cuis))
#                 novel_candidate_cuis.extend(filtered_cuis)
#                 novel_candidate_cuis_nomenclatures.extend(list_cuis_nomenclatures)
            
#             # If no valid CUIs were found for any entity in the feature, add to no_entities_list
#             if not feature_has_valid_cui:
#                 no_entities_list.append(f)
#         else:
#             # If no entities are found in the feature name, add to no_entities_list
#             no_entities_list.append(f)
    
#     # After processing all feature names, remove duplicates from no_entities_list
#     no_entities_list = list(set(no_entities_list))
#     print(f"{len(no_entities_list)}:\n No Entity feats: {no_entities_list}")
    
#     # Create the DataFrame with collected data
#     df_hits = pd.DataFrame({
#         "feature_name": novel_cols_candidates_names,
#         "cui": novel_candidate_cuis,
#         "cui_nomenclature": novel_candidate_cuis_nomenclatures,
#         "cui_def": list_cui_definitions,
#         "KG_Hits": list_cui_kg_hits,
#         "TUI": TUIs_list
#     }).drop_duplicates()

#     ##############################################################
#     ## merge with icu_feature_terms[["raw_name","name"]]
#     s1 = df_hits.shape[0]
#     df_hits = df_hits.loc[~df_hits["cui_nomenclature"].isin(REMOVE_CUI_TERMS_LIST)]
#     print(s1 - df_hits.shape[0], "Rows of unwanted cuis dropped")
#     for c in list_target_cuis:  ## manually append it hewre with some of the feature vals
#         df_hits._append({"cui": c, "feature_name": "target", "cui_nomenclature": linker.kb.cui_to_entity[c][1]}, ignore_index=True)
#     s1 = df_hits.shape[0]
#     # print(s1, "# rows pre semantic sim filt")
#     ### semantic similarity - heuristic, remove poor scoring pairs (by semantic similarity).
#     # ### Note:Could also expandthis with the CUIsdescriptions
#     # df_hits = get_sentence_pairs_similarity(df=df_hits, col1="cui_nomenclature", col2="feature_name", filter=True, minFilterValue=0.98 * sem_similarity_threshhold_score
#     #                                         , model2Name=None)

#     # df_hits.drop(columns=["v", "kg_hits_robust"], errors="ignore", inplace=True)
#     # "keep features where at least 1 potentially novel cui = unmatched in known literature-KG:"
#     # # df_hits = df_hits.query("feature_level_min_kg_hits==0 & feature_level_avg_kg_hits<0.7")
#     df_hits["cui"] = df_hits["cui"].astype(str)
#     df_hits.drop_duplicates(inplace=True)
#     print(df_hits[["feature_name", "cui"]].nunique())
#     print("# KG Hits:")
#     print(df_hits.query("KG_Hits>0")[["feature_name", "cui"]].nunique())
#     ## remove top hits filter 
#     ###### Add/Keep features that had 0 hits in the KG as additional candidate novels , for next stage of filtering
#     # * Add pseudovals for cui

#     ## TODO: Add in raw_name
#     df_hits = pd.concat([df_hits, pd.DataFrame({"feature_name": no_entities_list,
#                                                 "KG_Hits": [0] * len(no_entities_list),
#                                                 "cui_nomenclature": [""] * len(no_entities_list),
#                                                 "cui_def": [""] * len(no_entities_list),
#                                                 "cui": [""] * len(no_entities_list),
#                                                 "sim_score": [1] * len(no_entities_list)})], ignore_index=True)
    
#     ## add:
#     """Not enough! Doesn't handle empty features being added if present? """
#     df_hits.drop_duplicates(subset=["feature_name","cui","KG_Hits"],inplace=True) ## added
    
#     for c in df_hits.select_dtypes("number").columns:
#         # print(c)
#         df_hits[c] = df_hits[c].fillna(0)

#     # #### Rejoin with features metadata
#     # * * Warning: missing feature proxies won't be idd correctly may replace the version of column without missings.
#     #
#     # In[42]:

#     # In[43]:
#     df_hits = df_hits.merge(icu_feature_terms[['name', 'feature_importance', 'p_val', 'corr', "MutualInfoTarget",
#                                                'raw_name', # restore
#                                                ]].round(3),
#                             left_on=["feature_name"], right_on="name", how="left", validate="m:1").drop(columns=["name", "TUI"], errors="ignore")

#     # ### Very common/reoccurring nomenclatures - may be too broad
#     # * could remove these based on counts, tf-idf, percentile distribution.

#     print(f"# {len(no_entities_list)}features with no linked entities in them:\n")
#     print(len(novel_cols_candidates_names), "# novel candidate")
    
#     # print(f"{100*(len(novel_cols_candidates_names)/len(icu_feature_terms)):.2f}% candidates novel")
#     # for gallstones - 52% (78) when using TF linker, vs 66% (100) using statistical linker
#     # In[58]:
#     print("novel candidates # CUIS:", len(novel_candidate_cuis))


#     # # #### Add seperate sim score between feature name (+- cui?) and the TARGET
#     # # * NOTE! This differs from the OTHER sim_score (which was used for filtering NEL results); this one is more for further
#     # # In[38]:
#     # df_temp = df_hits[["feature_name", "cui_nomenclature"]].copy()  # .head(10) # .drop_duplicates(subset=["feature_name"])
#     # df_temp["target_name"] = TARGET_NAME
#     # df_hits["sim_score_target_feat"] = get_sentence_pairs_similarity(df=df_temp.copy(), col1="target_name", col2="feature_name", filter=False,
#     #                                                                  return_score_only=True)
#     # # df_hits["sim_score_target_cui"] = get_sentence_pairs_similarity(df=df_temp.copy(),col1="cui_nomenclature",col2="target_name",filter=False,return_score_only=True)
#     # df_hits["sim_score_target_cui"] = get_sentence_pairs_similarity(df=df_temp.copy(), col1="target_name", col2="cui_nomenclature",
#     #                                                                 model2Name=None,
#     #                                                                 filter=False, return_score_only=True)

#     # In[59]:
#     for c in list_target_cuis:
#         print(linker.kb.cui_to_entity[c][1])

#     # ## Graph connectivity metric
#     df_path_lengths = get_kg_connections(df_hits, df_kg, list_target_cuis)
#     print(df_path_lengths.shape,"df_path_lengths")
#     print(df_hits.shape, "df_hits")
#     # In[63]:
#     df_hits = df_hits.merge(df_path_lengths.drop(columns=["node_degree"], errors="ignore").drop_duplicates(), on="cui", how="left")

#     # ### Save output report
#     # * Saves highly filtered (heuristic) candiadtes, eg.g with no kg hits. (+- path length?)
#     # In[66]:
#     if SAVE_OUTPUTS:
#         df_hits["cui"] = df_hits["cui"].astype(str)
#         print(CANDIDATE_NOVEL_CUIS_FILEPATH)

#     if return_df:
#         return df_hits

In [6]:
def get_kg_connections(df_hits, df_kg, list_target_cuis):
    # ## Graph connectivity metric
    # * Get shortest path (could also get # simplest paths) between source and target, from graph.
    # * We'll calculate that only on pairs lacking a direct link to save compute + other tricks.
    #
    # *    "shortest_path_length": Higher is novel
    # *    "simple_path_length": Lower is novel
    # *    "norm_path_length": Lower is novel\
    #
    df_kg = df_kg.copy()
    ## make string to suppress replace warning:
    df_kg["SUBJECT_CUI"] = df_kg["SUBJECT_CUI"].astype(str)
    df_kg["OBJECT_CUI"] = df_kg["OBJECT_CUI"].astype(str)
    df_kg["SUBJECT_CUI"] = df_kg["SUBJECT_CUI"].replace(list_target_cuis, "y")
    df_kg["OBJECT_CUI"] = df_kg["OBJECT_CUI"].replace(list_target_cuis, "y")
    G = df_kg[["SUBJECT_CUI", "OBJECT_CUI"]].drop_duplicates().copy()
    # G["SUBJECT_CUI"] = G["SUBJECT_CUI"].replace(list_target_cuis, "y")
    # G["OBJECT_CUI"] = G["OBJECT_CUI"].replace(list_target_cuis, "y")
    G = G.drop_duplicates().reset_index(drop=True)
    assert G["SUBJECT_CUI"].value_counts()["y"] > 5
    assert G["OBJECT_CUI"].value_counts()["y"] > 5
    G.columns = ['source', 'target']
    # Create an undirected graph from the DataFrame:
    G = nx.from_pandas_edgelist(G, 'source', 'target', create_using=nx.Graph())
    ## make directed graph
    # G = nx.from_pandas_edgelist(G, 'source', 'target', create_using=nx.DiGraph())  # what about to-from target vs from-to ??
    ## drop prefilt of distance 0 cases , because we also want their simple paths
    df_temp = df_hits.drop_duplicates(subset=["cui"]).copy()  # .iloc[0:50,0:5]
    # # could skip and later readd all the cases with a known direct link to skip their compute time, if a bottleneck
    # df_known_cuis_list = df_temp.loc[df_temp["KG_Hits"]>0][["cui"]].reset_index(drop=True).copy()
    # df_known_cuis_list["short_path_length"] = 1 # shortest distance in nx - 1 dge
    # df_temp = df_temp.loc[df_temp["KG_Hits"]<1].reset_index(drop=True).copy()
    candidate_cuis_list = df_temp["cui"].unique()
    short_path_length_list = []
    # simple_path_length_list = []  ## simple paths part is much slkower!!
    # degrees_list = []  # store # out degree of nodes, (could do out degree if using directed graph ) for possible normalizing of simple paths?
    for i, c in tqdm(enumerate(candidate_cuis_list), total=len(candidate_cuis_list), desc="Calculating Paths"):
        if c in G:
            # degrees_list.append(G.degree(c))
            try:
                # Calculate shortest path length
                path_length = nx.shortest_path_length(G, source=c, target="y")
                short_path_length_list.append(path_length)
            except nx.NetworkXNoPath:
                # Handle cases where no path exists
                short_path_length_list.append(999)  # Indicate no path (smaller is better)  - float('inf') , or a placeholder large int
            # try:
            #     simple_path_length = len(list(nx.all_simple_edge_paths(G, source=c, target="y", cutoff=2)))  # can be slow!!
            #     simple_path_length_list.append(simple_path_length)
            # except:
            #     simple_path_length_list.append(0)
        else:
            # # Handle cases where the candidate node is not in the graph
            # print(f"Node {c} not found in the graph.")
            short_path_length_list.append(99)  # Use  float('inf') None or another marker (0) to indicate missing node
            # simple_path_length_list.append(0)
            # degrees_list.append(0)
    df_path_lengths = pd.DataFrame({"cui": candidate_cuis_list,
                                    "shortest_path_length": short_path_length_list,
                                    # "simple_path_length": simple_path_length_list,
                                    # "node_degree": degrees_list
                                    }).sort_values(by=["shortest_path_length"], ascending=True).drop_duplicates(subset=["cui"],
                                                                                                                keep="first")  # added drop dupes here and sorting
    # # lower is more novel
    # ## Disable this and it's constituents above
    # df_path_lengths["norm_path_length"] = (df_path_lengths["simple_path_length"].div(df_path_lengths["node_degree"]).fillna(0)).round(2)
    return df_path_lengths

In [7]:
## TODO: Add 2d degree distance linkage? +- Sem dist min filter
def temporal_kg_concepts_eval(df_hits,df_kg_sep,KG_YEAR_CUTOFF = 2013):
    """
    Foward Temporal eval
    ## Check which links/features from our data appear in "future" data
    * Take KG, split to train/test by time (e.g. 2014 onwards), if a link appears in 2014 onwads and not in earlier , then it is a possible case of a novel detection from our model.
    * Need to consider how to filter cuis/features. e.g. overly common cui nomenclatures

    :param df_hits:
    :param df_kg_sep:
    :param KG_YEAR_CUTOFF:
    :return:
    """

    print(KG_YEAR_CUTOFF,"KG_YEAR_CUTOFF")
    df_hits["cui_count"] = df_hits.groupby(["cui"]).transform("size")
    print(df_kg_sep.nunique())

    df_kg_past = df_kg_sep.loc[df_kg_sep["first_year_pair"]<KG_YEAR_CUTOFF]
    print("past",df_kg_past.shape[0])
    df_kg_future = df_kg_sep.loc[df_kg_sep["first_year_pair"]>=KG_YEAR_CUTOFF]
    print("Future, before filter/antijoin",df_kg_future.shape[0])


    df_kg_future = anti_join_df(left=df_kg_future, right=df_kg_past, key=["SUBJECT_CUI","OBJECT_CUI"])

    past_cuis_list = list(set(df_kg_past["SUBJECT_CUI"].unique().tolist() + df_kg_past["OBJECT_CUI"].unique().tolist()))
    print("# past cuis",len(past_cuis_list))

    future_cuis_list = list(set(df_kg_future["SUBJECT_CUI"].unique().tolist() + df_kg_future["OBJECT_CUI"].unique().tolist()))
    print("# future cuis",len(future_cuis_list))
    future_cuis_list = [x for x in future_cuis_list if x not in past_cuis_list]
    print("# future cuis, not in past",len(future_cuis_list))


    # ## Keep only (model) features that appear in the future, not past , of the KG
    #
    # * Additional possible filter needed: If any cui (perfeature) is a match in PAST data?
    #     * Would need to filter out overly common cuis (e.g. "genetic risk") in such a case or multiples...
    # * Q: Important filter note: Might want to filter by first appearance/year of a specific cui? e.g. to avoid cases where cui was only added after cutoff. This is arguable, as a novelty might legitiamtely be added as a cui after cutoff..

    df_hits_future = df_hits.loc[df_hits["cui"].isin(future_cuis_list)].copy()
    print(df_hits_future.shape[0], "# Rows")
    print("nunique:")
    # print(df_hits_future.groupby(["SUBJECT_CUI","OBJECT_CUI"]).size(), "# Subj X Obj cui unique Pairs")
    print(df_hits_future.nunique())
    print("\nfeature #:")
    print(df_hits_future.raw_name.value_counts().head(11))
    print("\ncui #:")
    print(df_hits_future.cui_nomenclature.value_counts().head(11))


    # ### Get feature level (vs cui) level matched hits in past and then use that for filtering
    # * Will take higher confidence cuis/terms (to reduce noise)
    # * We remove very common cuis that are too broad (e.g. "gene risk"). Set at 97% quantile (5 in our case for gallstones)

    # In[78]:

    df_hits_past = df_hits.loc[df_hits["cui"].isin(past_cuis_list)].copy()
    print(df_hits_past.shape[0],"# rows\n")
    print(df_hits_past.select_dtypes(["O","category"]).nunique(),"\n")

    ## redo cui count to use past data, avoid future information leak. Note: distribution looks about the same.
    df_hits_past["cui_count"] = df_hits_past.groupby(["cui"]).transform("size")

    # print(df_hits_past["cui_count"].describe(percentiles=[.5,.9,.97]).round(1))
    display(df_hits_past.drop_duplicates(["cui"])["cui_count"].describe(percentiles=[.5,.9,.95,.97]).round(1)) # avoid double counting cuis
    past_cui_count_cutoff = max(df_hits_past.drop_duplicates(["cui"])["cui_count"].quantile(0.97),2)
    print("cui cutoff for past:",past_cui_count_cutoff)
    df_hits_past = df_hits_past.loc[df_hits_past["cui_count"]<past_cui_count_cutoff]
    print("\nAfter cui cutoff filter:\n")
    print(df_hits_past.shape[0],"# rows\n")
    print(df_hits_past.select_dtypes(["O","category"]).nunique(),"\n")

    # ### Get results - # features in future and not in past, out of all utility featues
    # * Additional filter - rmeoval of "common" cui terms from the future results also (not just past).

    # In[80]:

    print(f'Out of {df_hits.query("KG_Hits>0")["raw_name"].nunique()} Features with any hit in KG')
    # print(df_hits_future["raw_name"].nunique(),"# future feats pre past filt")
    df_hits_future = df_hits_future.loc[(~df_hits_future["cui"].isin(df_hits_past["cui"])) & (~df_hits_future["raw_name"].isin(df_hits_past["raw_name"]))]
    print(df_hits_future["raw_name"].nunique(),"# future feats after past filt")


    # * Filter out common cuis like with past
    # * Use overall count instead of future only count of those cuis
    # * This filtering may be excessive?

    # In[81]:

    df_hits_future = df_hits_future.loc[df_hits_future["cui_count"]<=past_cui_count_cutoff]
    print(df_hits_future.drop_duplicates("raw_name").shape[0])
    display(df_hits_future.drop_duplicates("raw_name"))


In [8]:
# if __name__ == "__main__":
#     ## gallstones / Cholelithiasis
#     config = {
#       #   "FEATURES_REPORT_PATH":"../../gallstone_chol_ipw_feature_report.csv",
#       # "CANDIDATE_NOVEL_CUIS_FILEPATH":"../../candidate_novel_cuis_chol.csv",
#         "FEATURES_REPORT_PATH": "gallstone_chol_ipw_feature_report.csv",
#         "CANDIDATE_NOVEL_CUIS_FILEPATH": "candidate_novel_cuis_chol.csv", # output path
#             "TARGET_NAME" :"GALLSTONES, Cholelithiasis",
#             "additional_target_cui_terms_list" : ["C0008325", "C0008311"] }

#     df_hits = link_kg_concepts_slim(FEATURES_REPORT_PATH=config["FEATURES_REPORT_PATH"], CANDIDATE_NOVEL_CUIS_FILEPATH=config["CANDIDATE_NOVEL_CUIS_FILEPATH"],
#                      TARGET_NAME=config["TARGET_NAME"],additional_target_cui_terms_list=config["additional_target_cui_terms_list"],
#                                SAVE_OUTPUTS = False,
#                         MIN_EVIDENCE_FILTER=2,
#                                FAST=False
#                      )

#### Prep function for summarizing coverage
* how many covered at each stage , and in future
* Note - may be differences due to usage of extraction of all NERs here. (More noisy)

In [9]:
# !pip uninstall 'spacy[transformers]' -y
# !pip install scispacy spacy sentence_transformers -U

In [10]:
from configs import *#  config_celiac,config_gall,  config_gout

## import from configs

if FastRun:
    all_configs = [config_eye_occ,config_gall]
else:
    all_configs = [
    config_gall,  config_gout,config_celiac,
      config_spine, config_oesophagus, config_heart, config_eye_occ,config_depression
              ] 
config = all_configs[0]
nlp = spacy.load("en_core_sci_lg")
# nlp = spacy.load("en_core_sci_scibert")
nlp.add_pipe("abbreviation_detector")
nlp.add_pipe("scispacy_linker",
             config={"resolve_abbreviations": True,
                     "linker_name": "umls"
                     ,"max_entities_per_mention": 3  # 3, #6, #4, #5
                 ,"threshold": 0.88  ## default is 0.8, paper mentions 0.99 as thresh
                     })

/home/ddofer/anaconda3/envs/Medrag/lib/python3.11/site-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


In [46]:
%%time
all_results = []
for config in all_configs:
    results = {}
    print("--"*50,"\n", config["TARGET_NAME"],"\n","--"*30)
    # df_hits = link_kg_concepts_slim # ORIG run
    df_hits = link_kg_concepts(FEATURES_REPORT_PATH=config["FEATURES_REPORT_PATH"], CANDIDATE_NOVEL_CUIS_FILEPATH=config["CANDIDATE_NOVEL_CUIS_FILEPATH"],
                 TARGET_NAME=config["TARGET_NAME"],additional_target_cui_terms_list=config["additional_target_cui_terms_list"],
                    MIN_EVIDENCE_FILTER=4 if FastRun else 2,
                               # FAST=FastRun,
                               SAVE_OUTPUTS = False)

    icu_feature_terms = pd.read_csv(config["FEATURES_REPORT_PATH"])
    # icu_feature_terms = icu_feature_terms.loc[(icu_feature_terms["feature_importance"] > 0) |(icu_feature_terms["MutualInfoTarget"] >= 0.01)|(icu_feature_terms["p_val"] <= 0.05) ]  # filter a bit - optionally
    print(icu_feature_terms["name"].nunique(),"# unique names before cleaning")
    icu_feature_terms = wrangle_df_icu(icu_feature_terms,filter_feat_imp=True)
    print(icu_feature_terms["name"].nunique(),"# unique names post-cleaning") # considerable drop after cleaning up feature names, missing values, etc'

    ## note: Not using raw/cleaned so there may be disparity! can sanity check by looking at "no features list " 
    ## coverage of features with any linked KG entity
    num_inputFeat = icu_feature_terms["name"].nunique()
    num_covered_feat = df_hits["raw_name"].nunique()
    num_features_not_covered = num_inputFeat- num_covered_feat
    print("num_inputFeat:",num_inputFeat,"num_covered_feat:",num_covered_feat,"num_features_not_covered:",num_features_not_covered)
    
    ## direct link, ignores 1-2 hops:
    print((df_hits.groupby("raw_name")["KG_Hits"].max()>0).agg(["mean","sum","size","count"]).round(2))
    ## % cases/features with a link with targets in KG
    mean_kg_known = (100*(df_hits.groupby("raw_name")["KG_Hits"].max()>0).mean()).round(2)
    print("mean_kg_known:",mean_kg_known)
    
    ## inclde 1-2 hops with target  
    frac_dist2_linked = (100*(df_hits.groupby("raw_name")["shortest_path_length"].min()<=2).mean()).round(2)
    print("Distance 2 or less:",frac_dist2_linked)
    ## distance 1/direct only
    frac_dist1_linked = (100*(df_hits.groupby("raw_name")["shortest_path_length"].min()<=1).mean()).round(2)
    print("Distance 1 or less:",frac_dist1_linked)
    
    df_sub = df_hits.query("(shortest_path_length<=1) & (KG_Hits>0)").drop_duplicates(["raw_name","feature_name","cui"])
    print(f"df_sub ({df_sub.shape[0]} cases, with direct kg hits)")
    print(df_sub.select_dtypes("O").nunique())
    # display(df_sub)
    print()
    TARGET_NAME=config["TARGET_NAME"]
    additional_target_cui_terms_list=config["additional_target_cui_terms_list"]
    entity = test_nel(TARGET_NAME,nlp=nlp)
    # #### Target terms - may manually change
    list_target_cuis = [i[0] for i in entity._.kb_ents]
    if len(additional_target_cui_terms_list) > 0:
        list_target_cuis = list(set(list_target_cuis + additional_target_cui_terms_list))
    print("list_target_cuis",list_target_cuis)
    df_kg = pd.read_parquet("../../SemMed/predications.parquet",columns=['SUBJECT_CUI',  'OBJECT_CUI', 'first_year_pair'])
    df_kg["SUBJECT_CUI"] = df_kg["SUBJECT_CUI"].astype(str) # silence replace deprecation warnings
    df_kg["OBJECT_CUI"] = df_kg["OBJECT_CUI"].astype(str)
    df_kg["SUBJECT_CUI"] = df_kg["SUBJECT_CUI"].replace(list_target_cuis, "y")
    df_kg["OBJECT_CUI"] = df_kg["OBJECT_CUI"].replace(list_target_cuis, "y")
    # df_kg_sep = df_kg.loc[(df_kg["SUBJECT_CUI"].isin(list_target_cuis)) | (df_kg["OBJECT_CUI"].isin(list_target_cuis))].drop_duplicates().copy()
    df_kg_sep = df_kg.loc[(df_kg["SUBJECT_CUI"]=="y") | (df_kg["OBJECT_CUI"]=="y")].drop_duplicates().copy()
    # df_related = df_kg_sep[ df_kg_sep["SUBJECT_CUI"].isin(filtered_cuis) |  df_kg_sep["OBJECT_CUI"].isin(filtered_cuis)]
    df_kg_sep = df_kg_sep.groupby(["SUBJECT_CUI","OBJECT_CUI"],observed=True).min().reset_index()
    print("joined with dates, df_kg_sep:")
    print(df_kg_sep.shape[0])
    print(df_kg_sep.nunique())
    
    print("--"*20,TARGET_NAME)
    df_dist1_times = df_sub.merge(df_kg_sep.set_index("SUBJECT_CUI")["first_year_pair"],
                 left_on="cui",right_index=True,
                 how="left").merge(df_kg_sep.set_index("OBJECT_CUI")["first_year_pair"],left_on="cui",right_index=True,how="left")
    
    df_dist1_times["first_year_pair"] = df_dist1_times["first_year_pair_x"].fillna(df_dist1_times["first_year_pair_y"])
    df_dist1_times.dropna(subset=["first_year_pair"],axis=0,inplace=True)
    
    print(f"Post cutoff ({cutoffYear}) features' first appearance stats; :")
    df_dates = df_dist1_times.groupby("raw_name")["first_year_pair"].min().reset_index().drop_duplicates()
    df_dates["post_cutoff"] = (df_dates["first_year_pair"]>cutoffYear).astype(int)
    display(df_dates["post_cutoff"].agg(["mean","sum","size"]).round(2))
    print("--"*40)
    # Save results for the current target
    results[TARGET_NAME] = {
        "target":TARGET_NAME,
        "num_unique_input_features": num_inputFeat,
        "num_features_without_KG_link": num_features_not_covered, # appears in kg (regardless of target)
        "num_features_with_KG_link": num_covered_feat,# appears in kg (regardless of target)
        "percent_features_with_KG_link": mean_kg_known,
        "percent_features_within_1_hop": frac_dist1_linked,
        "num_features_within_1_hop":(df_hits.groupby("raw_name")["shortest_path_length"].min()<=1).sum(),
        "percent_features_within_2_hop": frac_dist2_linked,
        "num_features_within_2_hop":(df_hits.groupby("raw_name")["shortest_path_length"].min()<=2).sum(),
        "num_cases_post_cutoff": df_dates["post_cutoff"].sum(),
        "percent_cases_after_cutoff_year": df_dates["post_cutoff"].mean(),
        "list_of_target_cuis": list_target_cuis,
        "num_unique_target_cuis": len(set(list_target_cuis)),
        "df_kg_sep_size":df_kg_sep.shape[0],
        "frac_postCutoffYear_dist1":df_dates["post_cutoff"].mean().round(2),
        "sum_postCutoffYear_dist1":df_dates["post_cutoff"].sum(),
        "count_before_CutoffYear_dist1":(df_dates["post_cutoff"].size - df_dates["post_cutoff"].sum()),      
    }
    
    # Append results for this target to the all_results list
    all_results.append(results[TARGET_NAME])
    print("--"*40)

# Convert all_results into a DataFrame
df_results = pd.DataFrame(all_results).round(2)

---------------------------------------------------------------------------------------------------- 
 Esophageal cancer 
 ------------------------------------------------------------
After filtering KG min count
12915192
TARGET_NAME (For Entity-KG linking Esophageal cancer
All ents (Esophageal cancer,)
Esophageal cancer C0014859

--------------------------

Name:  Esophageal cancer
CUI: C0014859, Name: Esophageal Neoplasms
Definition: Tumors or cancer of the ESOPHAGUS.
TUI(s): T191
Aliases (abbreviated, total: 39): 
	 Esophagus Neoplasm, Neoplasm, Esophagus, Esophageal Tumors, Neoplasm, Esophageal, Esophagus Tumor, Tumor of the Esophagus, esophageal tumor, esophagus neoplasm, Neoplasms, Esophagus, Esophagus--Tumors
CUI: C0152018, Name: Esophageal carcinoma
Definition: A malignant epithelial tumor arising from the esophageal mucosa. Two major histologic types of esophageal carcinoma have been described: squamous cell carcinoma and adenocarcinoma. This type of cancer is associated with 

Calculating Paths: 100%|███████████████████████████████████████████████████████████████████████████| 402/402 [04:01<00:00,  1.66it/s]


(402, 5) df_path_lengths
(918, 19) df_hits
611 # unique names before cleaning
(611, 11) # feats preclean
(611, 11) # feats after mini filt and load
482 # unique names post-cleaning
num_inputFeat: 482 num_covered_feat: 410 num_features_not_covered: 72
mean       0.08
sum       32.00
size     410.00
count    410.00
Name: KG_Hits, dtype: float64
mean_kg_known: 7.8
Distance 2 or less: 97.8
Distance 1 or less: 7.8
df_sub (36 cases, with direct kg hits)
feature_name             32
cui                      19
cui_nomenclature         19
cui_def                  19
raw_name                 32
F.Split-Feature Split    32
dtype: int64

Esophageal cancer C0014859
list_target_cuis ['C0014859', 'C0152018', 'C0546837']
joined with dates, df_kg_sep:
1367
SUBJECT_CUI        1149
OBJECT_CUI          219
first_year_pair      75
dtype: int64
---------------------------------------- Esophageal cancer
Post cutoff (2012) features' first appearance stats; :


mean     0.31
sum     10.00
size    32.00
Name: post_cutoff, dtype: float64

--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
CPU times: user 5min 25s, sys: 9.16 s, total: 5min 34s
Wall time: 5min 35s


In [47]:
# ## pre replace name, raw_feature name
# %%time
# all_results = []
# for config in all_configs:
#     results = {}
#     print("--"*50,"\n", config["TARGET_NAME"],"\n","--"*30)
#     df_hits = link_kg_concepts_slim(FEATURES_REPORT_PATH=config["FEATURES_REPORT_PATH"], CANDIDATE_NOVEL_CUIS_FILEPATH=config["CANDIDATE_NOVEL_CUIS_FILEPATH"],
#                  TARGET_NAME=config["TARGET_NAME"],additional_target_cui_terms_list=config["additional_target_cui_terms_list"],
#                     MIN_EVIDENCE_FILTER=4 if FastRun else 2,
#                                FAST=FastRun, SAVE_OUTPUTS = False)

#     icu_feature_terms = pd.read_csv(config["FEATURES_REPORT_PATH"])
#     # icu_feature_terms = icu_feature_terms.loc[(icu_feature_terms["feature_importance"] > 0) |(icu_feature_terms["MutualInfoTarget"] >= 0.01)|(icu_feature_terms["p_val"] <= 0.05) ]  # filter a bit - optionally
#     print(icu_feature_terms["name"].nunique(),"# unique names before cleaning")
#     icu_feature_terms = wrangle_df_icu(icu_feature_terms,filter_feat_imp=True)
#     print(icu_feature_terms["name"].nunique(),"# unique names post-cleaning") # considerable drop after cleaning up feature names, missing values, etc'

#     ## note: Not using raw/cleaned so there may be disparity! can sanity check by looking at "no features list " 
#     ## coverage of features with any linked KG entity
#     num_inputFeat = icu_feature_terms["name"].nunique()
#     num_covered_feat = df_hits["feature_name"].nunique()
#     num_features_not_covered = num_inputFeat- num_covered_feat
#     print("num_inputFeat:",num_inputFeat,"num_covered_feat:",num_covered_feat,"num_features_not_covered:",num_features_not_covered)
    
#     ## direct link, ignores 1-2 hops:
#     print((df_hits.groupby("feature_name")["KG_Hits"].max()>0).agg(["mean","sum","size","count"]).round(2))
#     ## % cases/features with a link with targets in KG
#     mean_kg_known = (100*(df_hits.groupby("feature_name")["KG_Hits"].max()>0).mean()).round(2)
#     print("mean_kg_known:",mean_kg_known)
    
#     ## inclde 1-2 hops with target  
#     frac_dist2_linked = (100*(df_hits.groupby("feature_name")["shortest_path_length"].min()<=2).mean()).round(2)
#     print("Distance 2 or less:",frac_dist2_linked)
#     ## distance 1/direct only
#     frac_dist1_linked = (100*(df_hits.groupby("feature_name")["shortest_path_length"].min()<=1).mean()).round(2)
#     print("Distance 1 or less:",frac_dist1_linked)
    
#     df_sub = df_hits.query("(shortest_path_length<=1) & (KG_Hits>0)").drop_duplicates(["raw_name","feature_name","cui"])
#     print(f"df_sub ({df_sub.shape[0]} cases, with direct kg hits)")
#     print(df_sub.select_dtypes("O").nunique())
#     # display(df_sub)
#     print()
#     TARGET_NAME=config["TARGET_NAME"]
#     additional_target_cui_terms_list=config["additional_target_cui_terms_list"]
#     entity = test_nel(TARGET_NAME,nlp=nlp)
#     # #### Target terms - may manually change
#     list_target_cuis = [i[0] for i in entity._.kb_ents]
#     if len(additional_target_cui_terms_list) > 0:
#         list_target_cuis = list(set(list_target_cuis + additional_target_cui_terms_list))
#     print("list_target_cuis",list_target_cuis)
#     df_kg = pd.read_parquet("../../SemMed/predications.parquet",columns=['SUBJECT_CUI',  'OBJECT_CUI', 'first_year_pair'])
#     df_kg["SUBJECT_CUI"] = df_kg["SUBJECT_CUI"].astype(str) # silence replace deprecation warnings
#     df_kg["OBJECT_CUI"] = df_kg["OBJECT_CUI"].astype(str)
#     df_kg["SUBJECT_CUI"] = df_kg["SUBJECT_CUI"].replace(list_target_cuis, "y")
#     df_kg["OBJECT_CUI"] = df_kg["OBJECT_CUI"].replace(list_target_cuis, "y")
#     # df_kg_sep = df_kg.loc[(df_kg["SUBJECT_CUI"].isin(list_target_cuis)) | (df_kg["OBJECT_CUI"].isin(list_target_cuis))].drop_duplicates().copy()
#     df_kg_sep = df_kg.loc[(df_kg["SUBJECT_CUI"]=="y") | (df_kg["OBJECT_CUI"]=="y")].drop_duplicates().copy()
#     # df_related = df_kg_sep[ df_kg_sep["SUBJECT_CUI"].isin(filtered_cuis) |  df_kg_sep["OBJECT_CUI"].isin(filtered_cuis)]
#     df_kg_sep = df_kg_sep.groupby(["SUBJECT_CUI","OBJECT_CUI"],observed=True).min().reset_index()
#     print("joined with dates, df_kg_sep:")
#     print(df_kg_sep.shape[0])
#     print(df_kg_sep.nunique())
    
#     print("--"*20,TARGET_NAME)
#     df_dist1_times = df_sub.merge(df_kg_sep.set_index("SUBJECT_CUI")["first_year_pair"],
#                  left_on="cui",right_index=True,
#                  how="left").merge(df_kg_sep.set_index("OBJECT_CUI")["first_year_pair"],left_on="cui",right_index=True,how="left")
    
#     df_dist1_times["first_year_pair"] = df_dist1_times["first_year_pair_x"].fillna(df_dist1_times["first_year_pair_y"])
#     df_dist1_times.dropna(subset=["first_year_pair"],axis=0,inplace=True)
    
#     print(f"Post cutoff ({cutoffYear}) features' first appearance stats; :")
#     df_dates = df_dist1_times.groupby("raw_name")["first_year_pair"].min().reset_index().drop_duplicates()
#     df_dates["post_cutoff"] = (df_dates["first_year_pair"]>cutoffYear).astype(int)
#     display(df_dates["post_cutoff"].agg(["mean","sum","size"]).round(2))
#     print("--"*40)
#     # Save results for the current target
#     results[TARGET_NAME] = {
#         "target":TARGET_NAME,
#         "num_unique_input_features": num_inputFeat,
#         "num_features_without_KG_link": num_features_not_covered, # appears in kg (regardless of target)
#         "num_features_with_KG_link": num_covered_feat,# appears in kg (regardless of target)
#         "percent_features_with_KG_link": mean_kg_known,
#         "percent_features_within_1_hop": frac_dist1_linked,
#         "num_features_within_1_hop":(df_hits.groupby("feature_name")["shortest_path_length"].min()<=1).sum(),
#         "percent_features_within_2_hop": frac_dist2_linked,
#         "num_features_within_2_hop":(df_hits.groupby("feature_name")["shortest_path_length"].min()<=2).sum(),
#         "num_cases_post_cutoff": df_dates["post_cutoff"].sum(),
#         "percent_cases_after_cutoff_year": df_dates["post_cutoff"].mean(),
#         "list_of_target_cuis": list_target_cuis,
#         "num_unique_target_cuis": len(set(list_target_cuis)),
#         "df_kg_sep_size":df_kg_sep.shape[0],
#         "frac_postCutoffYear_dist1":df_dates["post_cutoff"].mean().round(2),
#         "sum_postCutoffYear_dist1":df_dates["post_cutoff"].sum(),
#         "count_before_CutoffYear_dist1":(df_dates["post_cutoff"].size - df_dates["post_cutoff"].sum()),      
#     }
    
#     # Append results for this target to the all_results list
#     all_results.append(results[TARGET_NAME])
#     print("--"*40)

# # Convert all_results into a DataFrame
# df_results = pd.DataFrame(all_results).round(2)

In [48]:
# df_results = pd.read_csv("./Outputs/Figures/future_recall_coverage_report.csv")

In [58]:
df_dates.drop_duplicates("raw_name").sort_values("first_year_pair")

,raw_name,first_year_pair,post_cutoff
29,Wheeze or whistling in the chest in last year_Yes,1953.0,0
10,Long-term/recurrent antibiotics as child or te...,1957.0,0
9,Long-term/recurrent antibiotics as child or te...,1957.0,0
28,Weight change during worst episode of depressi...,1977.0,0
11,"Major dietary changes in the last 5 years_Yes,...",1977.0,0
26,Salt added to food_Usually,1982.0,0
24,Salt added to food_Never/rarely,1982.0,0
25,Salt added to food_Sometimes,1982.0,0
7,Haemoglobin concentration,1983.0,0
8,Hyperplasia of prostate,1987.0,0


In [59]:
df_sub[['feature_name', 'cui', 'cui_nomenclature', 'KG_Hits', 'raw_name']].columns#.drop_duplicates("cui_nomenclature")

Index(['feature_name', 'cui', 'cui_nomenclature', 'cui_def', 'KG_Hits',
       'sim_score', 'feature_level_min_kg_hits', 'feature_level_sum_kg_hits',
       'feature_level_avg_kg_hits', 'feature_importance', 'p_val', 'corr',
       'MutualInfoTarget', 'raw_name', 'F.Split-Lift (y==1)',
       'F.Split-Support', 'F.Split-Feature Split', 'sim_score_target_feat',
       'sim_score_target_cui', 'shortest_path_length', 'simple_path_length',
       'norm_path_length'],
      dtype='object')

In [67]:
df_dist1_times[['feature_name',  'cui_nomenclature', 'KG_Hits', 'raw_name',"first_year_pair","shortest_path_length"]].query("first_year_pair>2010").drop_duplicates("feature_name")#.nunique()

,feature_name,cui_nomenclature,KG_Hits,raw_name,first_year_pair,shortest_path_length
14,Method of diagnosis when first had COVID-19,COVID19 (disease),1,Method of diagnosis when first had COVID-19_None,2020.0,1
39,Vitamin D,ergocalciferol,1,Vitamin D,2012.0,1
238,Concentration of Very Small VLDL Particles,Particle,1,Concentration of Very Small VLDL Particles,2020.0,1
370,Salt added to food Sometimes,Salts,1,Salt added to food_Sometimes,2020.0,1
436,Method of diagnosis when first had COVID-19 Co...,COVID19 (disease),1,Method of diagnosis when first had COVID-19_Co...,2020.0,1
540,"Blood clot, DVT, bronchitis, emphysema, asthma...",Hypersensitivity,1,"Blood clot, DVT, bronchitis, emphysema, asthma...",2020.0,1
569,"Blood clot, DVT, bronchitis, emphysema, asthma...",Hypersensitivity,1,"Blood clot, DVT, bronchitis, emphysema, asthma...",2020.0,1
579,"Blood clot, DVT, bronchitis, emphysema, asthma...",Hypersensitivity,1,"Blood clot, DVT, bronchitis, emphysema, asthma...",2020.0,1
593,"Blood clot, DVT, bronchitis, emphysema, asthma...",Hypersensitivity,1,"Blood clot, DVT, bronchitis, emphysema, asthma...",2020.0,1
621,Salt added to food Usually,Salts,1,Salt added to food_Usually,2020.0,1


In [51]:
display(df_results)

,target,num_unique_input_features,num_features_without_KG_link,num_features_with_KG_link,percent_features_with_KG_link,percent_features_within_1_hop,num_features_within_1_hop,percent_features_within_2_hop,num_features_within_2_hop,num_cases_post_cutoff,percent_cases_after_cutoff_year,list_of_target_cuis,num_unique_target_cuis,df_kg_sep_size,frac_postCutoffYear_dist1,sum_postCutoffYear_dist1,count_before_CutoffYear_dist1
0,Esophageal cancer,482,72,410,7.8,7.8,32,97.8,401,10,0.31,"[C0014859, C0152018, C0546837]",3,1367,0.31,10,22


In [52]:
if SAVE:
    df_results.to_csv("./Outputs/Figures/future_recall_coverage_report.csv",index=False)

#### Get negative candidates
* feat imp
* Negative interesting + make fae explanation
* maybe check that model didn't find as interesting ? 

In [53]:
icu_feature_terms = pd.read_csv(config["FEATURES_REPORT_PATH"])

icu_weak  = wrangle_df_icu(icu_feature_terms,filter_feat_imp=False)

icu_weak = icu_feature_terms.loc[(icu_weak["feature_importance"]< 0.0001) &(icu_feature_terms["MutualInfoTarget"] < 0.0001)] 
# icu_weak

(611, 11) # feats preclean


In [54]:
df_dist1_times.nunique()

feature_name                 32
cui                          19
cui_nomenclature             19
cui_def                      19
KG_Hits                       3
sim_score                    35
feature_level_min_kg_hits     2
feature_level_sum_kg_hits     3
feature_level_avg_kg_hits     8
feature_importance           17
p_val                        24
corr                         27
MutualInfoTarget             23
raw_name                     32
F.Split-Lift (y==1)          25
F.Split-Support              30
F.Split-Feature Split        32
sim_score_target_feat        19
sim_score_target_cui         21
shortest_path_length          1
simple_path_length           12
norm_path_length             10
first_year_pair_x            11
first_year_pair_y             5
first_year_pair              14
dtype: int64

In [55]:
df_dates["post_cutoff"].describe().round(3)

count    32.000
mean      0.312
std       0.471
min       0.000
25%       0.000
50%       0.000
75%       1.000
max       1.000
Name: post_cutoff, dtype: float64

In [56]:
df_sub["cui_nomenclature"].value_counts().head(20)

cui_nomenclature
Hypersensitivity                      4
Minerals                              4
COVID19 (disease)                     4
Food                                  3
Salts                                 3
Chronic Obstructive Airway Disease    2
Adolescent (age group)                2
Diabetes Mellitus                     2
Diet                                  2
Coffee                                1
Hyperplasia                           1
Peptic Esophagitis                    1
Pleural effusion disorder             1
Particle                              1
Hemoglobin                            1
Chest                                 1
vitamin D                             1
ergocalciferol                        1
Screening procedure                   1
Name: count, dtype: int64